5.6.2배치용 Affine 계층
    - 데이터 N개를 묶은 데이터인 '배치'로 순전파하는 경우
    - 5.6.1의 예에서 X의 형상이 (N, 2)로 변한 것 외 동일

    - aL/aX = aL/aY *W^T == (N,2) = (N,3) * (3,2)
    - aL/aW = X^T * aL/aY == (2,3) = (2,N) * (N,3)
    - aL/aB = aL/aY == (3) = (N,3)

In [3]:
# 순전파의 경우, 편향은 N개의 데이터에 각각 더해짐

import numpy as np

X_dot_W=np.array([[0, 0, 0], [10, 10, 10]])
B=np.array([1, 2, 3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [4]:
# 역전파의 경우, 각 데이터의 역전파 값이 편향의 원소에 모여야 함

dY=np.array([[1, 2, 3], [4, 5, 6]])
print(dY)

dB=np.sum(dY, axis=0)   # N개의 데이터에 대한 미분을 데이터마다 더해서 연산 ==0번째 축에 대한 총합 연산
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [5]:
# Affine 구현

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x =x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db - np.sum(dout, axis=0)

        return dx

5.6.3 Softmax-with-Loss 계층
    - 출력층에서 사용하는 softmax 함수
    - 교차 엔트로피 오차 포함한 소프트맥스 계층

    - Softmax: 입력 값을 정규화하여 출력 (출력의 합이 1이 되도록 변형)

    - 추론 시에, 사용 X/ 마지막 Affine 계층의 출력을 인식 결과로 이용
    - 점수(score): Softmax 앞의 Affine 계층의 출력

    - softmax는 주로 학습 시에 활용 

    [교재 표기]
    - an: 입력
    - yn: 입력 an을 softmax로 정규화한 출력값
    - tn: 정답 레이블
    - L: softmax 계층의 출력과 정답 레이블을 입력으로 받아, cross entropy error 계층으로부터 손실 출력

    - softmax의 역전파: softmax 계층 출력과 정답 레이블의 차분
    == 신경망의 역전파에서 오차가 앞 계층에 전달되는 것을 의미
    - but, 있는 그대로 전달하는 것은 오차를 그대로 드러내는 것

    ex)
    - 정답 레이블: (0, 1, 0)
    - softmax 계층: (0.3, 0.2, 0.5)
    - 역전파: (0.3, -0.8, 0.5)

    - 정답 레이블: (0, 1, 0)
    - softmax 계층: (0.01, 0.99, 0)
    - 역전파: (0.01, -0.01, 0)
    -> 오차가 작아졌으니, 학습하는 정도도 줄어듬

In [7]:
# softmax 구현(3.5.2 참고)

def softmax(a):
    c=np.max(a)
    exp_a=np.exp(a-c) # 오버플로 방지
    sum_exp_a=np.sum(exp_a)
    y=exp_a/sum_exp_a
    
    return y

In [8]:
# cross_entropy_error() 구현(4.2.2 참고)

def cross_entropy_error(y, t):
    delta=1e-7
    return -np.sum(t*np.log(y+delta))

In [10]:
# softmax 구현

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None        # 손실
        self.y= None            # softmax의 출력
        self.t = None           # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    # 역전파의 경우, 전파하는 값을 배치의 수로 나누어 데이터 1개당 오차를 앞 계층으로 전파
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx